In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import scipy

# Clean Carbon Data

Data taken from https://www.eex.com/en/market-data/environmental-markets/eua-primary-auction-spot-download . Accessed on 20/10/2022.

Read the excel file containing the Carbon Price data.

In [2]:
read_carbon_path = Path("__file__").parent.joinpath("Raw_Data", "Carbon_Data").parent.joinpath("Carbon_Data", "Carbon Price Data.xlsx")
df = pd.read_excel(read_carbon_path, index_col=False)

Clean the data by removing any duplicates, dropping unnecessary columns, adding in all missing dates (this will be used later) and renaming the columns to easier names to write.

In [3]:
df = df.drop_duplicates(subset = "Date", keep = "first")
df["Date"] = pd.to_datetime(df['Date'])
df = (df.set_index('Date').reindex(pd.date_range('2012-01-01', '2022-02-24', freq='D')).rename_axis(['Date']).fillna(0).reset_index())
df = df.drop(df.columns[[2,3,4,5]], axis = 1)
df.rename(columns = {'Auction Price €/tCO2':'Carbon Price'}, inplace = True)
df.head()


,Date,Carbon Price
0,2012-01-01,0.00
1,2012-01-02,0.00
2,2012-01-03,6.31
3,2012-01-04,0.00
4,2012-01-05,0.00


Save the data to the manipulated data folder

In [4]:
write_carbon_path = Path("__file__").parent.joinpath("Manipulated_Data", "Carbon_Data").parent.joinpath("Carbon_Data", "Daily-Carbon Prices.csv")
df.to_csv(write_carbon_path)

# Clean Coal Data

##### Data taken from https://markets.businessinsider.com/commodities/coal-price. Accessed on 24/10/2022

Read csv file containing coal data

In [5]:
read_coal_path = Path("__file__").parent.joinpath("Raw_Data", "Coal_Data").parent.joinpath("Coal_Data", "Coal Data.csv")
df = pd.read_csv(read_coal_path, index_col= False)
#df.head()

Clean the data by removing unwanted columns, renaming and dropping any duplicates and reordering the rows.

In [6]:
df["Date"] = pd.to_datetime(df['Date'])
df = df.drop(df.columns[[2,3]], axis = 1)
df = (df.set_index('Date').reindex(pd.date_range('2012-01-01', '2022-02-24', freq='D')).rename_axis(['Date']).fillna(0).reset_index())


In [7]:
df = df.sort_values(by = "Date", ascending = True)
df = df.reindex(columns = ["Date", "Close"])
df.rename(columns = {'Close':'Coal Price'}, inplace = True)
df = df.drop_duplicates(subset = "Date", keep = "first")
df.head()

,Date,Coal Price
0,2012-01-01,0.00
1,2012-01-02,0.00
2,2012-01-03,109.35
3,2012-01-04,109.55
4,2012-01-05,110.10


In [8]:
write_coal_path = Path("__file__").parent.joinpath("Manipulated_Data", "Coal_Data").parent.joinpath("Coal_Data", "Coal-Daily.csv")
df.to_csv(write_coal_path)

# Clean Oil Data

##### Data pulled from https://datahub.io/core/oil-prices#pandas

Read csv file containing oil data

In [9]:
read_oil_path = Path("__file__").parent.joinpath("Raw_Data", "Oil_Data").parent.joinpath("Oil_Data", "Brent Crude Data.csv")
df = pd.read_csv(read_oil_path)
df.head()

,Unnamed: 0.1,Date,Unnamed: 0,Price
0,0,2012-01-01,0.0,0.00
1,1,2012-01-02,0.0,0.00
2,2,2012-01-03,6251.0,111.12
3,3,2012-01-04,6252.0,113.37
4,4,2012-01-05,6253.0,113.59


Clean the data by removing unwanted columns, renaming and reordering the columns and dropping any duplicates.

In [10]:
df['Date'] = pd.to_datetime(df['Date'])
df = df[df.Date >= "01-01-2012"]  
df = (df.set_index('Date').reindex(pd.date_range('2012-01-01', '2022-02-24', freq='D')).rename_axis(['Date']).fillna(0).reset_index())
df = df.drop(df.columns[[1,2]], axis = 1)
df = df.drop_duplicates(subset = "Date", keep = "first")
df.rename(columns = {'Price':'Oil Price'}, inplace = True)
df.head()


,Date,Oil Price
0,2012-01-01,0.00
1,2012-01-02,0.00
2,2012-01-03,111.12
3,2012-01-04,113.37
4,2012-01-05,113.59


In [11]:
write_oil_path = Path("__file__").parent.joinpath("Manipulated_Data", "Oil_Data").parent.joinpath("Oil_Data", "Oil-Daily.csv")
df.to_csv(write_oil_path)

# Merge all the data into a single dataframe

Read all of the data into separate dataframes.

In [12]:
read_temp_path = Path("__file__").parent.joinpath("Manipulated_Data", "Weather_Data").parent.joinpath("Weather_Data", "Temperature Index 2.csv")
df1 = pd.read_csv(write_carbon_path, index_col = False)
df2 = pd.read_csv(write_coal_path, index_col = False)
df3 = pd.read_csv(write_oil_path, index_col = False)
df4 = pd.read_csv(read_temp_path, index_col = False)

Merge the carbon and coal data

In [13]:
df = pd.merge(df1, df2, how = 'outer', on = 'Date')
df = df.drop(df.columns[[0,3]], axis = 1)

Merge the oil data with the dataframe

In [14]:
df = pd.merge(df, df3, how = 'outer', on = 'Date')
df = df.drop(df.columns[[3]], axis = 1)

Merge the temperature index with the dataframe

In [15]:
df = pd.merge(df, df4, how = 'outer', on = 'Date')
df.tail()

,Date,Carbon Price,Coal Price,Oil Price,Weighted Temp
3703,2022-02-20,0.00,0.00,0.00,NaN
3704,2022-02-21,87.99,185.15,98.95,NaN
3705,2022-02-22,88.40,189.00,98.73,NaN
3706,2022-02-23,92.80,190.65,99.29,NaN
3707,2022-02-24,88.00,193.10,101.29,NaN


Remove all instances where the carbon price is zero and any places where he have an na value. This will ensure that all cells are filled with data points.

In [16]:
df = df[df["Carbon Price"] > 0]
df.dropna(inplace= True)

Set the data to only include after a certain date

In [17]:
#df = df[df.Date >= "2018-01-01"]
#df.head()

Export final dataframe to csv.

In [18]:
write_complete_dataframe_path = Path("__file__").parent.joinpath("Manipulated_Data", "Merged_Data").parent.joinpath("Merged_Data", "Merged-Data.csv")
df.to_csv(write_complete_dataframe_path)